# Collaborative Filtering
> A notebook that helps us to discover one of pioneered framework in Deep Learning - FastAI
- toc: true
- branch: master
- badges: true
- comments: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2
- image: https://developers.google.com/machine-learning/recommendation/images/1Dmatrix.svg
- description: Fourth in a series on understanding FastAI.